In [50]:
import cv2
import tifffile as tiff
import os
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [34]:
# img_2015 = cv2.imread('input/1_0_960_.jpg')

# plt.figure(figsize=(18, 5))
# plt.subplot(131)
# plt.hist(np.ravel(img_2015[:,:,0]))
# plt.subplot(132)
# plt.hist(np.ravel(img_2015[:,:,1]))
# plt.subplot(133)
# plt.hist(np.ravel(img_2015[:,:,2]))
# plt.show()

In [51]:
# 归一化
def scale_percentile_n(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix      

# 标准化
# def z_score(x, axis=(0, 1)):
#     x = np.array(x).astype(float)
#     return (x-np.mean(x, axis=axis))/np.std(x, axis=axis)

In [17]:
# FILE_2015 = '../preliminary/quickbird2015.tif'
# FILE_2017 = '../preliminary/quickbird2017.tif'
# FILE_2015 = '../preliminary/quickbird2015_preliminary_2.tif'
# FILE_2017 = '../preliminary/quickbird2017_preliminary_2.tif'
# img_2015 = tiff.imread(FILE_2015).transpose([1, 2, 0])
# img_2017 = tiff.imread(FILE_2017).transpose([1, 2, 0])
# print img_2015.shape

(3000, 15106, 4)


## 加载final图片

In [52]:
FILE_2015 = '../preliminary/quickbird2015.tif'
FILE_2017 = '../preliminary/quickbird2017.tif'
img_2015 = tiff.imread(FILE_2015).transpose([1, 2, 0])
img_2017 = tiff.imread(FILE_2017).transpose([1, 2, 0])
print img_2015.shape

(5106, 15106, 4)


In [53]:
# 设置通道数
channel = 3
img_2015 = img_2015[:,:,:channel]
img_2017 = img_2017[:,:,:channel]
print img_2015.shape

(5106, 15106, 3)


# 加权平均3通道做灰度代替4通道

In [44]:
# img_2015 = img_2015.astype(float)
# img_2017 = img_2017.astype(float)

# temp_15 = np.ones((5106, 15106, 5))
# temp_17 = np.ones((5106, 15106 ,5))

# temp_15[:, :, :4] = img_2015[:, :, :4]
# temp_17[:, :, :4] = img_2017[:, :, :4]
# temp_15[:, :, 4] = np.dot(img_2015[:, :, :3], [0.299, 0.587, 0.114])
# temp_17[:, :, 4] = np.dot(img_2017[:, :, :3], [0.299, 0.587, 0.114])

# img_2015 = temp_15
# img_2017 = temp_17

# print(img_2015.shape)

(5106, 15106, 5)


In [19]:
# img_size = 960

# for i in range(im_2015.shape[0]//img_size + 1): # last 284
#     for j in range(im_2015.shape[1]//img_size + 1): #last 2 too small, drop one
#         im_name = str(i)+'_'+str(j)+'_'+str(img_size)+'_.jpg'
#         tif_name = str(i)+'_'+str(j)+'_'+str(img_size)+'_.tif'
#         cv2.imwrite("input/satellite/2017/"+im_name,stretch_n(im_2017[i*img_size:i*img_size+img_size, j*img_size:j*img_size+img_size, :3])*255)
#         cv2.imwrite("input/satellite/2015/"+im_name,stretch_n(im_2015[i*img_size:i*img_size+img_size, j*img_size:j*img_size+img_size, :3])*255)
#         tiff.imsave("input/satellite/2017/"+tif_name,np.array(stretch_n(im_2017[i*img_size:i*img_size+img_size, j*img_size:j*img_size+img_size, :3])*255, dtype=np.uint8))
#         tiff.imsave("input/satellite/2015/"+tif_name,np.array(stretch_n(im_2015[i*img_size:i*img_size+img_size, j*img_size:j*img_size+img_size, :3])*255, dtype=np.uint8))
#         cv2.imwrite("input/satellite/cada/"+im_name,im_cada[i*img_size:i*img_size+img_size, j*img_size:j*img_size+img_size]*255)
#         cv2.imwrite("input/satellite/tiny/"+im_name,im_tiny[i*img_size:i*img_size+img_size, j*img_size:j*img_size+img_size]*255)

# 将原图切割出960x960的图像并单独做归一化后保存

In [54]:
#将图片切割成960x960
IM_ROWS = 5106
# IM_ROWS = 4000
# IM_ROWS = 3000
IM_COLS = 15106
ROI_SIZE = 960

DATA_DIR = 'input'

if not os.path.exists(DATA_DIR + '/train960'):
    os.makedirs(DATA_DIR + '/train960')

for i in range(int(IM_ROWS // ROI_SIZE)+1):
    for j in range(int(IM_COLS // ROI_SIZE)+1):
        ss2 = '{}/train960/{}_{}_{}_{}.tif'.format(DATA_DIR, 2015, i, j, ROI_SIZE)
        if os.path.exists(ss2):
            continue
        tiff.imsave(ss2, np.array(scale_percentile_n(img_2015[i*ROI_SIZE:(i+1)*ROI_SIZE,j*ROI_SIZE:(j+1)*ROI_SIZE])*255,dtype=np.uint8))
#         tiff.imsave(ss2, z_score(img_2015[i*ROI_SIZE:(i+1)*ROI_SIZE,j*ROI_SIZE:(j+1)*ROI_SIZE]))
    
for i in range(int(IM_ROWS // ROI_SIZE)+1):
    for j in range(int(IM_COLS // ROI_SIZE)+1):
        ss2 = '{}/train960/{}_{}_{}_{}.tif'.format(DATA_DIR, 2017, i, j, ROI_SIZE)
        if os.path.exists(ss2):
            continue
        tiff.imsave(ss2, np.array(scale_percentile_n(img_2017[i*ROI_SIZE:(i+1)*ROI_SIZE,j*ROI_SIZE:(j+1)*ROI_SIZE])*255,dtype=np.uint8))
#         tiff.imsave(ss2, z_score(img_2017[i*ROI_SIZE:(i+1)*ROI_SIZE,j*ROI_SIZE:(j+1)*ROI_SIZE]))

print('Generation completed')

Generation completed


In [55]:
# 生成训练用图像
DATA_DIR = 'input'
years = [2015, 2017]
# years = [2015]
cut_size = 160
spacing = 120
channel = 3 #[0,1,2]
if not os.path.exists(DATA_DIR+'/train'):
    os.makedirs(DATA_DIR+'/train')
if not os.path.exists(DATA_DIR+'/label'):
    os.makedirs(DATA_DIR+'/label')
# if not os.path.exists(DATA_DIR+'/labeltxt'):
#     os.makedirs(DATA_DIR+'/labeltxt')
count = 0
flag = 0
if cut_size%spacing != 0:
    flag = 1
for year in years:
    for i in range(int(IM_ROWS // ROI_SIZE + 1)):
        for j in range(int(IM_COLS // ROI_SIZE + 1)):
            ss2 = '{}/train960/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, ROI_SIZE)
            ss2_label = '{}/mylabel_{}/{}_{}_{}_.jpg'.format(DATA_DIR, year, i, j, ROI_SIZE)
            if os.path.exists(ss2) and os.path.exists(ss2_label):
                img = tiff.imread(ss2)
                img_label = cv2.imread(ss2_label, -1)
                for x in range(int(ROI_SIZE // spacing - (cut_size//spacing + flag - 1))):
                    for y in range(int(ROI_SIZE // spacing - (cut_size//spacing + flag - 1))):
                        ss3 = '{}/train/{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y)
                        ss3_label = '{}/label/{}_{}_{}_{}_{}.jpg'.format(DATA_DIR, year, i, j, x, y)
                        ss3_txt = '{}/labeltxt/{}_{}_{}_{}_{}.txt'.format(DATA_DIR, year, i, j, x, y)
                        img_temp = img[x*spacing:(x*spacing+cut_size), y*spacing:(y*spacing+cut_size)]
                        if img_temp.shape != (cut_size, cut_size, channel):
                            continue
                        tiff.imsave(ss3, img_temp)
                        cv2.imwrite(ss3_label,img_label[x*spacing:(x*spacing+cut_size), y*spacing:(y*spacing+cut_size)])
#                         np.savetxt(ss3_txt, img_label[x*spacing:(x*spacing+cut_size),y*spacing:(y*spacing+cut_size),0] > 128, fmt='%d')
                        count += 1
#                         image = tiff.imread(ss3)
#                         if image is None:
#                             os.remove(ss3)
#                         else:
#                             if (image.shape[0]==cut_size) and (image.shape[1]==cut_size):
#                                 count += 1
#                                 cv2.imwrite(ss3_label,img_label[x*spacing:(x*spacing+cut_size), y*spacing:(y*spacing+cut_size)])
#                                 np.savetxt(ss3_txt,img_label[x*spacing:(x*spacing+cut_size),y*spacing:(y*spacing+cut_size),0] > 128, fmt='%d')
#                             else:
#                                 os.remove(ss3)

print('Generation completed! Create ', str(count), 'pictures')

('Generation completed! Create ', '7770', 'pictures')


In [7]:
# 切割测试用图像（已弃用，整图切成小图然后进行归一化操作，效果不好）
import tifffile as tiff
import os
import cv2

IM_ROWS = 5106
IM_COLS = 15106
ROI_SIZE = 160

DATA_DIR = 'input'
if not os.path.exists(DATA_DIR+'/result_image'):
    os.makedirs(DATA_DIR+'/result_image')
years = [2015, 2017]
for year in years:
    img = eval('img_{}'.format(year))
    print(img.sum())
    for i in range(int(IM_ROWS // ROI_SIZE)): 
        for j in range(int(IM_COLS // ROI_SIZE)): 
            ss2 = '{}/result_image/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, ROI_SIZE)
            if os.path.exists(ss2):
                continue
            tiff.imsave(ss2, scale_percentile_n(img[i*ROI_SIZE:(i+1)*ROI_SIZE, j*ROI_SIZE:(j+1)*ROI_SIZE]))

    # 把下面i+1行的边角图片向上扩增成ROI_SIZE的图片
    if IM_ROWS%ROI_SIZE != 0:
        i = i+1
        for k in range(int(IM_COLS // ROI_SIZE)):
            ss2 = '{}/result_image/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, k, ROI_SIZE)
            if os.path.exists(ss2):
                continue
            tiff.imsave(ss2, scale_percentile_n(img[(IM_ROWS-ROI_SIZE):IM_ROWS, k*ROI_SIZE:(k+1)*ROI_SIZE]))
    # 把右面j+1列的边角图片向左扩增成ROI_SIZE的图片
    if IM_COLS%ROI_SIZE != 0:
        j = j+1
        for k in range(int(IM_ROWS // ROI_SIZE)):
            ss2 = '{}/result_image/{}_{}_{}_{}.tif'.format(DATA_DIR, year, k, j, ROI_SIZE)
            if os.path.exists(ss2):
                continue
            tiff.imsave(ss2, scale_percentile_n(img[k*ROI_SIZE:(k+1)*ROI_SIZE, (IM_COLS-ROI_SIZE):IM_COLS]))
    # 把右下角的图片扩增成ROI_SIZE的图片
    if IM_ROWS%ROI_SIZE != 0 and IM_ROWS%ROI_SIZE != 0:
        ss2 = '{}/result_image/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, ROI_SIZE)
        if os.path.exists(ss2):
            continue
        tiff.imsave(ss2, scale_percentile_n(img[(IM_ROWS-ROI_SIZE):IM_ROWS, (IM_COLS-ROI_SIZE):IM_COLS]))
    
print('Generation completed')

103217710447
86692180482
Generation completed
